In [ ]:
import requests
import pandas as pd
import arrow
import datetime
from datetime import timezone
import os
import quandl
import numpy as np
from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
def get_quote_data(symbol='AAPL', data_range='2y', data_interval='1h'):
    res = requests.get('https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?range={data_range}&interval={data_interval}'.format(**locals()))
    data = res.json()
    body = data['chart']['result'][0]    
    dt = datetime.datetime

    dt = pd.Series(map(lambda x: arrow.get(x).datetime.replace(tzinfo=None), body['timestamp']), name='Datetime')
    df = pd.DataFrame(body['indicators']['quote'][0], index=dt)
    dg = pd.DataFrame(body['timestamp'])    
    df = df.loc[:, ('open', 'high', 'low', 'close', 'volume')]
    df.dropna(inplace=True)     #removing NaN rows
    df.columns = ['OPEN', 'HIGH','LOW','CLOSE','VOLUME']    #Renaming columns in pandas

    return df

In [ ]:
final_df = get_quote_data()
final_df['Ticker'] = 'AAPL'
list_of_tickers = ['GOOGL','MSFT','AMZN','FB','BABA','JNJ','JPM','BAC','WMT']
for ticker in list_of_tickers:
    stock_df = get_quote_data(symbol=ticker)
    stock_df['Ticker'] = ticker
    final_df = pd.concat([final_df,stock_df])
final_df.head()

In [ ]:
final_df.head(10)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 5, 5
r = final_df
date_list = np.array(r.index.to_pydatetime())
# print(date_list)
plot_array = np.zeros([len(r), 5])
plot_array[:, 0] = np.arange(plot_array.shape[0])
plot_array[:, 1:] = r.iloc[:, :4]

# print(plot_array[init:init+7])
# print(len(plot_array))
init = 0
while(init<(len(plot_array)-7)):
# while(init<(15)):
    fig, ax = plt.subplots(nrows=1, ncols=1)
    num_of_bars = 7  # the number o
    open_list = plot_array[init:init+7][:,1]
    close_list = plot_array[init:init+7][:,4]
    ax.margins(x=0.0, y=0.1)
    ax.yaxis.tick_right()
    x_tick_labels = []
#     print(plot_array[6, 0])
    ax.set_xlim(right=plot_array[6, 0])
    ax.grid(True, color='k', ls='--', alpha=0.2)
#     print("hiii",plot_array[init,0],plot_array[init+7, 0])
    indices = np.linspace(plot_array[0,0], plot_array[6, 0], 7, dtype=int)
    x_tick_labels = ['14:30', '15:30', '16:30', '17:30', '18:30', '19:30', '20:30']
    ax.set(xticks = indices,xticklabels=x_tick_labels)
#     print(x_tick_labels,lst1)
    ax.plot(x_tick_labels,open_list)
    ax.plot(x_tick_labels,close_list)
    plot_array[init:init+7][:,0] = [0., 1., 2., 3., 4., 5., 6.]
    candlestick_ohlc(ax, plot_array[init:init+7], colorup='g', colordown='r')
    plt.show()
    fig.savefig('new_images_2/'+str(init)+'.png',dpi=20)
    init+=7
